In [1]:
!pip install transformers > /dev/null 2>&1
!pip install datasets "transformers[sentencepiece]" > /dev/null 2>&1
!pip install sentencepiece > /dev/null 2>&1
!pip install bitsandbytes > /dev/null 2>&1
!pip install accelerate -U > /dev/null 2>&1
!pip install --upgrade jupyterlab ipywidgets > /dev/null 2>&1
!pip install evaluate > /dev/null 2>&1
!pip install optimum
!pip install onnxruntime
!pip install onnx

print("Hello, pip installation is done")

Hello, pip installation is done


In [2]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv('datasets/cli_natural_language_command_dataset.csv')

dataset = Dataset.from_pandas(df)

print(dataset)
print(df.head())



Dataset({
    features: ['input', 'output'],
    num_rows: 900
})
                                   input  \
0    Create a green pyramid at 68 -50 -6   
1      Create a yellow cube at -75 33 41   
2     Create a green sphere at 16 54 -15   
3      Create a red pyramid at 70 90 -46   
4  Create a yellow pyramid at 31 -80 -52   

                                    output  
0    create-object pyramid green 68 -50 -6  
1      create-object cube yellow -75 33 41  
2     create-object sphere green 16 54 -15  
3      create-object pyramid red 70 90 -46  
4  create-object pyramid yellow 31 -80 -52  


In [3]:

from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_function(examples):
    model_inputs = tokenizer(examples['input'], max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(examples['output'], max_length=128, truncation=True, padding='max_length')
    
    # Store the labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Step to shuffle the dataset before processing
shuffled_dataset = dataset.shuffle(seed=42)  # You can set a seed for reproducibility
# Tokenize the dataset
tokenized_dataset = shuffled_dataset.map(tokenize_function, batched=True)
# Remove non-tokenized columns to clean the dataset
tokenized_dataset = tokenized_dataset.remove_columns(['input', 'output'])


# Split the dataset into 80% train and 20% test
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)

# Access the train and test sets
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Verify the sizes
print(f"Train size: {len(train_dataset)}, Test size: {len(eval_dataset)}")
print(tokenized_dataset[0])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Train size: 720, Test size: 180
{'input_ids': [3, 28141, 22734, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [9268, 18, 30536, 22734, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [4]:
from sklearn.metrics import accuracy_score
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Convert predictions and labels to text (if using text generation models like T5)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Optionally, log the decoded predictions and labels
    for i in range(5):  # Log the first 5 predictions
        print(f"Prediction: {decoded_preds[i]}")
        print(f"Label: {decoded_labels[i]}")
        print('-' * 30)
    
    # Calculate your metrics (example: accuracy for simple tasks)
    acc = accuracy_score(decoded_labels, decoded_preds)
    
    return {"accuracy": acc}

In [5]:
from transformers import TrainingArguments,Seq2SeqTrainer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments

model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

training_args = Seq2SeqTrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    # save_total_limit=3,
    num_train_epochs=7,
    predict_with_generate=True,
    logging_dir='./logs',
    fp16=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.315338,0.000000
2,No log,0.071643,0.000000
3,No log,0.009145,0.772222
4,No log,0.004513,0.938889
5,No log,0.003119,0.977778
6,0.672800,0.002424,1.000000
7,0.672800,0.002221,1.000000


/opt/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Prediction: 
Label: move-object cube 99 14 -58
------------------------------
Prediction: 
Label: set-texture sphere wood
------------------------------
Prediction: 
Label: delete-object pyramid
------------------------------
Prediction: 
Label: delete-object pyramid
------------------------------
Prediction: 
Label: set-transparency sphere 0.4
------------------------------


/opt/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Prediction: 
Label: move-object cube 99 14 -58
------------------------------
Prediction: 
Label: set-texture sphere wood
------------------------------
Prediction: 
Label: delete-object pyramid
------------------------------
Prediction: 
Label: delete-object pyramid
------------------------------
Prediction: 
Label: set-transparency sphere 0.4
------------------------------


...

/opt/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Prediction: move-object cube 99 14 -58
Label: move-object cube 99 14 -58
------------------------------
Prediction: set-texture sphere wood
Label: set-texture sphere wood
------------------------------
Prediction: delete-object pyramid
Label: delete-object pyramid
------------------------------
Prediction: delete-object pyramid
Label: delete-object pyramid
------------------------------
Prediction: set-transparency sphere 0.4
Label: set-transparency sphere 0.4
------------------------------


/opt/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Prediction: move-object cube 99 14 -58
Label: move-object cube 99 14 -58
------------------------------
Prediction: set-texture sphere wood
Label: set-texture sphere wood
------------------------------
Prediction: delete-object pyramid
Label: delete-object pyramid
------------------------------
Prediction: delete-object pyramid
Label: delete-object pyramid
------------------------------
Prediction: set-transparency sphere 0.4
Label: set-transparency sphere 0.4
------------------------------


TrainOutput(global_step=630, training_loss=0.5362921429058862, metrics={'train_runtime': 265.595, 'train_samples_per_second': 18.976, 'train_steps_per_second': 2.372, 'total_flos': 170530669854720.0, 'train_loss': 0.5362921429058862, 'epoch': 7.0})

In [7]:
from datetime import datetime

# Get current date and time
now = datetime.now()

# Format as 'Y-m-d H:i'
formatted_time = now.strftime('%Y_%m_%d_%H_%M')

# Assuming the model and tokenizer are already loaded and trained
model.save_pretrained("trained_models/" + formatted_time)
tokenizer.save_pretrained("trained_models/"  + formatted_time)

('trained_models/2024_10_20_02_19/tokenizer_config.json',
 'trained_models/2024_10_20_02_19/special_tokens_map.json',
 'trained_models/2024_10_20_02_19/spiece.model',
 'trained_models/2024_10_20_02_19/added_tokens.json')

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

input_text = "Create a blue pyramid at 25 -31 -79"

model_name_or_path = "trained_models/" + formatted_time #path/to/your/model/or/name/on/hub
device = "cpu" # or "cuda" if you have a GPU



trainedModel = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path).to(device)
trainedTokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

inputs = trainedTokenizer(input_text, return_tensors='pt', max_length=128, truncation=True, padding='max_length').to(device)
outputs = trainedModel.generate(inputs['input_ids'],     max_new_tokens=128)
print("Answer:{" + trainedTokenizer.decode(outputs[0], skip_special_tokens=True)+"}")


# After training, during evaluation or inference, check predictions
outputs = trainedModel.generate(
    trainedTokenizer("Create a cube at 10 20 30", return_tensors="pt").input_ids,
num_beams=4,    # Beam search to improve output quality
    early_stopping=True,
)
    # Stop once output is complete)

# Decode the model's predictions
decoded_output = trainedTokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Model Output: [{decoded_output}]")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Answer:{create-object pyramid blue 25 -31 -79}


/opt/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Model Output: [create-object cube 10 20 30]


In [12]:
!pip install onnx

from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

save_directory = "onnx/"

# Load a model from transformers and export it to ONNX
ort_model = ORTModelForSequenceClassification.from_pretrained(model_name_or_path, export=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
ort_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KeyError: 't5 is not supported yet with the onnx backend. Only [] are supported. If you want to support onnx please propose a PR or open up an issue.'

In [2]:
import onnx

# Load the ONNX model
model = onnx.load("trained_models/t5_onnx/encoder_model.onnx")

# Check the model for issues
onnx.checker.check_model(model)
print("The model is valid!")

# Print the opset version used in the model
print("Opset version:", model.opset_import[0].version)

The model is valid!
Opset version: 13
